In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline
def savefig(name):
    plt.savefig(name + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(name + ".pdf", bbox_inches='tight')

In [ ]:
reload(pu_delayfb)
carbons = [1,2,3,4,5,6,7]

single_C_decay_data = dict()

for c in carbons:
    data = dict()
    data["x"], data["y"], data["y_u"], data["fr"] = \
    pu_delayfb.number_of_repetitions_stitched(
        multi_contains = ['sweep_number_of_reps_C%d_X' % (c)],
        older_than="20170812000000",
        do_fit = True,
        fixed=[0,2,5,6,4], 
        ret_data_fit=True,
        T2star_correction=True,
        LDE_element_length=7e-6,
    )
    single_C_decay_data[c] = data

In [ ]:
reload(pu_delayfb)
fb_results = {}
tomo_list = ['X', 'Y']

for c in carbons:
    c_key = c
    fb_results[c_key] = {}

    folder_names = []
    for d_i in range(2):
        name_template = "phase_fb_delayline_C%d_sec%d" % (
            c,
            d_i
        )
        folder_names += [name_template]
    print folder_names
    for t in tomo_list:
        print("Tomo: %s" % t)
        data = dict()
        data["x"], data["y"], data["y_u"], data["fr"] = \
        pu_delayfb.number_of_repetitions_stitched(
            contains="",
            multi_contains=folder_names,
            older_than='20170816000000',
            do_fit=True,
            x_only=True,
            tomo_basis=t,
            fit_n=1.0,
            fixed=[0,2,5,6,4],
            ret_data_fit=True,
            T2star_correction=True,
            LDE_element_length=7e-6
        )
        fb_results[c_key][t] = data

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = plt.subplot()
i=0
fit_xvals = np.linspace(0.,1000., 2000.)
for c in carbons:
    data = single_C_decay_data[c]
    plt.errorbar(
        data["x"], data["y"], data["y_u"], 
        fmt='o', color="C%d" % i, 
        label = r"C%d, $N_{1/e} = %d \pm %d$" % (c, np.round(data["fr"]['params_dict']['T']),
                                                                 np.round(data["fr"]['error_dict']['T']))
    )
    plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color="C%d" % i)
    i += 1

plt.ylim(0,1)
plt.xlim(0,600)

plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Bloch vector length")
# plt.title("Single-nuclear state decay")

# ax.set_yscale('log', nonposy='clip')
# plt.ylim(0.001,1.0)

lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))    
# plt.savefig("batch-plots/single-nuclear-state-decay.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
savefig("thesis-plots/single-nuclear-state-decay")
plt.show()
plt.close('all')

In [ ]:
data["x"]

In [ ]:
fig = plt.figure(figsize=(4.4,4.2))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[3, 1])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
fit_xvals = np.linspace(0.,1000., 2000.)

colors = ["C%d" % d for d in range(6)] + ['0.5']

for i_c, c in enumerate(carbons):
    data = fb_results[c]['X']
    nofbdata = single_C_decay_data[c]
    eb = ax1.errorbar(
        data["x"], data["y"], data["y_u"], 
        fmt='o',
        color= colors[i_c],
        alpha=0.5,
        label = r"C%d, $N_{1/e} = %d \pm %d$" % (c, np.round(data["fr"]['params_dict']['T']),
                                                                 np.round(data["fr"]['error_dict']['T']))
    )
    ax1.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color=eb[0].get_color())
    
    eb2 = ax2.errorbar(
        data["x"]+i_c*3, data["y"] - nofbdata["y"], np.sqrt(nofbdata["y_u"]**2 + data["y_u"]**2),
        fmt='o',
        ms=3,
        alpha=0.5,
        color=eb[0].get_color()
    )
    i += 1

ax1.set_ylim(0,1)
ax1.set_xlim(0,600)
ax2.set_xlim(0,600)

# ax1.set_xlabel("Number of LDE attempts")
ax1.set_ylabel(r"$\langle X \rangle$")
# plt.title("Single-nuclear state decay")
ax2.set_xlabel("number of LDE attempts")
ax2.set_ylabel("Bloch vector\ndifference")
ax2.set_ylim(-0.2,0.2)

lgd = ax1.legend(loc=2, bbox_to_anchor=(1,1))    
# plt.savefig("batch-plots/single-nuclear-feedback-decay.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.tight_layout()
savefig("thesis-plots/single-nuclear-feedback-decay-w-residuals")
plt.show()
plt.close('all')

In [ ]:
diffs = []

for c in carbons:
    data = fb_results[c]['X']
    nofbdata = single_C_decay_data[c]
    diffs += [data["y"] - nofbdata["y"]]
    
diff_arr = np.concatenate(diffs)

In [ ]:
np.mean(diff_arr)

In [ ]:
len(diff_arr)

In [ ]:
np.std(diff_arr) / np.sqrt(len(diff_arr) - 1)

In [ ]:
for i_c in carbons: 
    print "carbon: ", i_c
    data = single_C_decay_data[i_c]
    print data["fr"]['params_dict']['A']
    print data["fr"]['error_dict']['A']
    print data["y"][0]
    print data["y_u"][0]

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)
import itertools

freqs = np.zeros((len(carbons), 3)) # -1, 0, +1
# T2stars = np.zeros((len(carbons) + 2*len(carbon_combis)))

c_idxs = dict()

for i_c, c in enumerate(carbons):
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    msp1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "p1")]
    msm1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "m1")]  
    freqs[i_c,0] = msm1_freq
    freqs[i_c,1] = ms0_freq
    freqs[i_c,2] = msp1_freq
    
# 
# +1 frequencies of C4 and C5 are interchanged
freqs[3:5,2] = freqs[3:5,2][::-1]

for i_c, c in enumerate(carbons):
    print("C%d dephasing rate parameter: %.3f kHz" % (c, 1e-3 * (freqs[i_c,2]-freqs[i_c,0])) )
    
    c_idxs[str(c)] = i_c
    
print freqs

In [ ]:
tau_naive = 30e-9
tau_singlet = 300e-9

for i_c, c in enumerate(carbons):
    N_naive = 4. / (tau_naive * 2*np.pi*(freqs[i_c, 0] - freqs[i_c, 1]))**2
    N_singlet = 4. / (tau_singlet * 2*np.pi*(freqs[i_c,0] - freqs[i_c,2]))**2
    
    print "carbon: ", c
    print "N naive: ", N_naive
    print "N singlet", N_singlet